In [ ]:
import unittest
import json
from typing import Dict, List, Optional, Union
from unittest import mock
from pydantic import BaseModel

from your_package.utils.gemini_utils import GeminiUtils


class TestModel(BaseModel):
    name: str
    age: int
    email: Optional[str] = None
    tags: List[str] = []


def sample_function(name: str, value: int, optional_param: str = "default") -> Dict[str, Union[str, int]]:
    """This is a sample function for testing."""
    return {"name": name, "value": value, "optional": optional_param}


class TestGeminiUtils(unittest.TestCase):
    def test_pydantic_to_schema_basic(self):
        schema = GeminiUtils.pydantic_to_schema(TestModel)
        self.assertEqual(schema["type"], "OBJECT")
        self.assertIn("properties", schema)
        self.assertIn("name", schema["properties"])
        self.assertEqual(schema["properties"]["name"]["type"], "STRING")
        self.assertEqual(schema["properties"]["age"]["type"], "INTEGER")
        
    def test_pydantic_to_schema_with_optional(self):
        schema = GeminiUtils.pydantic_to_schema(TestModel)
        self.assertIn("email", schema["properties"])
        self.assertIn("nullable", schema["properties"]["email"])
        
    def test_pydantic_to_schema_with_list(self):
        schema = GeminiUtils.pydantic_to_schema(TestModel)
        self.assertIn("tags", schema["properties"])
        self.assertEqual(schema["properties"]["tags"]["type"], "ARRAY")
        
    def test_pydantic_to_schema_non_pydantic(self):
        dict_schema = {"type": "OBJECT", "properties": {}}
        result = GeminiUtils.pydantic_to_schema(dict_schema)
        self.assertEqual(result, dict_schema)
        
    def test_python_to_function_basic(self):
        func_def = GeminiUtils.python_to_function(sample_function)
        self.assertEqual(func_def["name"], "sample_function")
        self.assertEqual(func_def["description"], "This is a sample function for testing.")
        self.assertIn("parameters", func_def)
        
    def test_python_to_function_parameters(self):
        func_def = GeminiUtils.python_to_function(sample_function)
        params = func_def["parameters"]
        self.assertEqual(params["type"], "OBJECT")
        self.assertIn("name", params["properties"])
        self.assertIn("value", params["properties"])
        self.assertIn("optional_param", params["properties"])
        
    def test_python_to_function_required_params(self):
        func_def = GeminiUtils.python_to_function(sample_function)
        required = func_def["parameters"]["required"]
        self.assertIn("name", required)
        self.assertIn("value", required)
        self.assertNotIn("optional_param", required)
        
    def test_python_to_function_custom_name(self):
        func_def = GeminiUtils.python_to_function(sample_function, name="custom_name")
        self.assertEqual(func_def["name"], "custom_name")
        
    def test_python_to_function_custom_description(self):
        func_def = GeminiUtils.python_to_function(sample_function, description="Custom description")
        self.assertEqual(func_def["description"], "Custom description")
        
    def test_python_type_to_schema_primitive(self):
        self.assertEqual(GeminiUtils._python_type_to_schema(str)["type"], "STRING")
        self.assertEqual(GeminiUtils._python_type_to_schema(int)["type"], "INTEGER")
        self.assertEqual(GeminiUtils._python_type_to_schema(float)["type"], "NUMBER")
        self.assertEqual(GeminiUtils._python_type_to_schema(bool)["type"], "BOOLEAN")
        
    def test_python_type_to_schema_list(self):
        schema = GeminiUtils._python_type_to_schema(List[str])
        self.assertEqual(schema["type"], "ARRAY")
        self.assertEqual(schema["items"]["type"], "STRING")
        
    def test_python_type_to_schema_dict(self):
        schema = GeminiUtils._python_type_to_schema(Dict)
        self.assertEqual(schema["type"], "OBJECT")
        
    def test_python_type_to_schema_optional(self):
        from typing import Optional
        schema = GeminiUtils._python_type_to_schema(Optional[int])
        self.assertEqual(schema["type"], "INTEGER")
        self.assertTrue(schema["nullable"])
        
    def test_functions_to_tools(self):
        def func1(a: int): pass
        def func2(b: str): pass
        
        tools = GeminiUtils.functions_to_tools([func1, func2])
        self.assertEqual(len(tools), 2)
        self.assertEqual(tools[0]["name"], "func1")
        self.assertEqual(tools[1]["name"], "func2")
        
    def test_extract_function_calls_empty(self):
        calls = GeminiUtils.extract_function_calls({})
        self.assertEqual(calls, [])
        
    def test_extract_function_calls_string(self):
        calls = GeminiUtils.extract_function_calls('{"function_calls": [{"name": "test", "args": {"x": 1}}]}')
        self.assertEqual(len(calls), 1)
        self.assertEqual(calls[0]["name"], "test")
        self.assertEqual(calls[0]["args"], {"x": 1})
        
    def test_extract_function_calls_functionCalls(self):
        response = {"functionCalls": [{"name": "test", "args": {"x": 1}}]}
        calls = GeminiUtils.extract_function_calls(response)
        self.assertEqual(len(calls), 1)
        self.assertEqual(calls[0]["name"], "test")
        
    def test_extract_function_calls_function_calls(self):
        response = {"function_calls": [{"name": "test", "args": {"x": 1}}]}
        calls = GeminiUtils.extract_function_calls(response)
        self.assertEqual(len(calls), 1)
        self.assertEqual(calls[0]["name"], "test")
        
    def test_extract_function_calls_candidates(self):
        response = {"candidates": [{"functionCall": {"name": "test", "args": {"x": 1}}}]}
        calls = GeminiUtils.extract_function_calls(response)
        self.assertEqual(len(calls), 1)
        self.assertEqual(calls[0]["name"], "test")
        
    def test_extract_function_calls_toolCalls(self):
        response = {"toolCalls": [{"function": {"name": "test", "arguments": '{"x": 1}'}}]}
        calls = GeminiUtils.extract_function_calls(response)
        self.assertEqual(len(calls), 1)
        self.assertEqual(calls[0]["name"], "test")
        self.assertEqual(calls[0]["args"], {"x": 1})
        
    def test_extract_function_calls_invalid_json(self):
        response = {"toolCalls": [{"function": {"name": "test", "arguments": "invalid-json"}}]}
        calls = GeminiUtils.extract_function_calls(response)
        self.assertEqual(len(calls), 1)
        self.assertEqual(calls[0]["name"], "test")
        self.assertEqual(calls[0]["args"], {})


if __name__ == '__main__':
    unittest.main()